# Analyze correlation of amino-acid fitnesses with dN/dS values
This notebook looks at how dN/dS values from FEL correlation with the amino-acid fitness values.

In [1]:
######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/SARS2-mut-fitness/lib/python3.9/site-packages', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmpdaq7171u/file/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xc0\x17\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c,results_public_2023-05-11/dnds/dnds_data.csv\x94\x8c3results_public_2023-05-11/aa_fitness/aa_fitness.csv\x94\x8c?results_public_2023-05-11/dms/dadonaite_ba1_spike/processed.csv\x94\x8c5results_public_2023-05-11/dms/starr_rbd/processed.csv\x94\x8c8results_public_2023-05-11/dms/iketani_mpro/processed.csv\x94\x8c;results_public_2023-05-11/dms/flynn_mpro_2022/processed.csv\x94\x8c;results_public_2023-05-11/dms/flynn_mpro_2023/processed.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x04dnds\x94K\x00N\x86\x94\x8c\naa_fitness\x94K\x01N\x86\x94\x8c\x13dadonaite_ba1_spike\x94K\x02N\x86\x94\x8c\tstarr_rbd\x94K\x03N\x86\x94\x8c\x0ciketani_mpro\x94K\x04N\x86\x94\x8c\x0fflynn_mpro_2022\x94K\x05N\x86\x94\x8c\x0fflynn_mpro_2023\x94K\x06N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh$\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h*)}\x94\x8c\x05_name\x94h$sNt\x94bh%h(h*\x85\x94R\x94(h*)}\x94h.h%sNt\x94bh\x14h\nh\x16h\x0bh\x18h\x0ch\x1ah\rh\x1ch\x0eh\x1eh\x0fh h\x10ub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8c-results_public_2023-05-11/dnds/dnds_corr.html\x94a}\x94(h\x12}\x94\x8c\tcorr_html\x94K\x00N\x86\x94sh"]\x94(h$h%eh$h(h*\x85\x94R\x94(h*)}\x94h.h$sNt\x94bh%h(h*\x85\x94R\x94(h*)}\x94h.h%sNt\x94bh;h8ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94(K\n}\x94(h\x18}\x94(\x8c\x05study\x94\x8c;https://www.biorxiv.org/content/10.1101/2022.10.13.512056v1\x94\x8c\x0bdescription\x94\x8c\x1dspike (Dadonaite et al, 2022)\x94\x8c\x04gene\x94\x8c\x01S\x94\x8c\x03url\x94\x8c\x8bhttps://raw.githubusercontent.com/dms-vep/SARS-CoV-2_Omicron_BA.1_spike_DMS_mAbs/main/results/muteffects_functional/muteffects_observed.csv\x94\x8c\x0bfilter_cols\x94}\x94\x8c\ntimes_seen\x94K\x03suh\x1a}\x94(\x8c\x05study\x94\x8cOhttps://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1010951\x94\x8c\x0bdescription\x94\x8c\x17RBD (Starr et al, 2022)\x94\x8c\x04gene\x94hQ\x8c\x03url\x94\x8c\xadhttps://media.githubusercontent.com/media/jbloomlab/SARS-CoV-2-RBD_DMS_Omicron/f76ba2b2bec18ede9fa9da18c9ccc52389b1ba3a/results/final_variant_scores/final_variant_scores.csv\x94uh\x1c}\x94(\x8c\x05study\x94\x8cChttps://www.sciencedirect.com/science/article/pii/S1931312822004024\x94\x8c\x0bdescription\x94\x8c\x1aMpro (Iketani et al, 2022)\x94\x8c\x04gene\x94\x8c\x0bnsp5 (Mpro)\x94\x8c\x03url\x94\x8c\xachttps://raw.githubusercontent.com/alejandrochavezlab/3CL_protease_DMS/dc802ee3ea9d43005ca97a28af078767fd66777c/outputs/results/normalized_to_wt_and_stop/activity_scores.csv\x94\x8c\x06wt_seq\x94\x8c\x0cdata/Mpro.fa\x94uh\x1e}\x94(\x8c\x05study\x94\x8c(https://elifesciences.org/articles/77433\x94\x8c\x0bdescription\x94\x8c\x18Mpro (Flynn et al, 2022)\x94\x8c\x04gene\x94\x8c\x0bnsp5 (Mpro)\x94\x8c\x03url\x94\x8cKhttps://cdn.elifesciences.org/articles/77433/elife-77433-fig2-data1-v2.xlsx\x94\x8c\x06wt_seq\x94\x8c\x0cdata/Mpro.fa\x94uh }\x94(\x8c\x05study\x94\x8c)https://doi.org/10.1101/2023.03.02.530652\x94\x8c\x0bdescription\x94\x8c\x18Mpro (Flynn et al, 2023)\x94\x8c\x04gene\x94\x8c\x0bnsp5 (Mpro)\x94\x8c\x03url\x94\x8cahttps://www.biorxiv.org/content/biorxiv/early/2023/03/02/2023.03.02.530652/DC1/embed/media-1.xlsx\x94uue}\x94(h\x12}\x94(\x8c\x12min_expected_count\x94K\x00N\x86\x94\x8c\x0cdms_datasets\x94K\x01N\x86\x94uh"]\x94(h$h%eh$h(h*\x85\x94R\x94(h*)}\x94h.h$sNt\x94bh%h(h*\x85\x94R\x94(h*)}\x94h.h%sNt\x94bh\x80K\nh\x82hJub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x11public_2023-05-11\x94a}\x94(h\x12}\x94\x8c\x03mat\x94K\x00N\x86\x94sh"]\x94(h$h%eh$h(h*\x85\x94R\x94(h*)}\x94h.h$sNt\x94bh%h(h*\x85\x94R\x94(h*)}\x94h.h%sNt\x94b\x8c\x03mat\x94h\x91ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\x12}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh"]\x94(h$h%eh$h(h*\x85\x94R\x94(h*)}\x94h.h$sNt\x94bh%h(h*\x85\x94R\x94(h*)}\x94h.h%sNt\x94bh\xa8K\x01h\xaaK\x01h\xach\xa5ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c1results_public_2023-05-11/dnds/analyze_dnds.ipynb\x94a}\x94(h\x12}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh"]\x94(h$h%eh$h(h*\x85\x94R\x94(h*)}\x94h.h$sNt\x94bh%h(h*\x85\x94R\x94(h*)}\x94h.h%sNt\x94bh\xbeh\xbbub\x8c\x06config\x94}\x94(\x8c\tmat_trees\x94}\x94(\x8c\x11public_2023-05-11\x94\x8c\x83http://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/2023/05/11/public-2023-05-11.all.masked.nextclade.pangolin.pb.gz\x94\x8c\x11public_2023-03-27\x94\x8c\x83http://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/2023/03/27/public-2023-03-27.all.masked.nextclade.pangolin.pb.gz\x94\x8c\x11public_2022-12-18\x94\x8c\x83http://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/2022/12/18/public-2022-12-18.all.masked.nextclade.pangolin.pb.gz\x94\x8c\x11gisaid_2023-03-29\x94\x8cQhttps://hgwdev.gi.ucsc.edu/~angie/2847b92/gisaidAndPublic.2023-03-29.masked.pb.gz\x94u\x8c\x0bcurrent_mat\x94\x8c\x11public_2023-05-11\x94\x8c\tref_fasta\x94\x8cGhttp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/bigZips/wuhCor1.fa.gz\x94\x8c\x07ref_gtf\x94\x8cPhttp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/bigZips/genes/ncbiGenes.gtf.gz\x94\x8c\x0eadd_to_ref_gtf\x94}\x94\x8c\x05ORF9b\x94]\x94(M|nM\xa1oes\x8c\x11min_clade_samples\x94M\x10\'\x8c\x0esample_subsets\x94}\x94(\x8c\x03all\x94\x8c\x01.\x94\x8c\x03USA\x94\x8c\x03USA\x94\x8c\x07England\x94\x8c\x07England\x94u\x8c\x12clade_founder_json\x94\x8cyhttps://raw.githubusercontent.com/neherlab/SC2_variant_rates/7e738194a8c6592082f1caa9a6ca70cb68289790/data/clade_gts.json\x94\x8c\x11clades_to_exclude\x94]\x94\x8c\x0321A\x94a\x8c\x10max_nt_mutations\x94K\x04\x8c\x15max_reversions_to_ref\x94K\x01\x8c\x1fmax_reversions_to_clade_founder\x94K\x01\x8c\x1bexclude_ref_to_founder_muts\x94\x88\x8c\x12site_include_range\x94}\x94(\x8c\x05start\x94M\n\x01\x8c\x03end\x94M\xeasu\x8c\x10sites_to_exclude\x94]\x94(K\x99M}\x04M\x96\x08MI\x0cM\xec\rM\xc2\x0eM\xd2\x0fMo\x18MV\x1fMV"MZ4M{6M\x12YME_MF_MeaM\xfd\x15M\xc3\x1aM\xa0\x1cM\xbfmM\xb2re\x8c\rsite_mask_vcf\x94\x8cvhttps://raw.githubusercontent.com/W-L/ProblematicSites_SARS-CoV2/master/subset_vcf/problematic_sites_sarsCov2.mask.vcf\x94\x8c\x12usher_masked_sites\x94\x8c\x1cdata/usher_masked_sites.yaml\x94\x8c\x1dsynonymous_spectra_min_counts\x94M\x88\x13\x8c\x0eorf1ab_to_nsps\x94}\x94(\x8c\x04nsp1\x94]\x94(K\x01K\xb4e\x8c\x04nsp2\x94]\x94(K\xb5M2\x03e\x8c\x04nsp3\x94]\x94(M3\x03M\xcb\ne\x8c\x04nsp4\x94]\x94(M\xcc\nM\xbf\x0ce\x8c\x0bnsp5 (Mpro)\x94]\x94(M\xc0\x0cM\xf1\re\x8c\x04nsp6\x94]\x94(M\xf2\rM\x13\x0fe\x8c\x04nsp7\x94]\x94(M\x14\x0fMf\x0fe\x8c\x04nsp8\x94]\x94(Mg\x0fM,\x10e\x8c\x04nsp9\x94]\x94(M-\x10M\x9d\x10e\x8c\x05nsp10\x94]\x94(M\x9e\x10M(\x11e\x8c\x0cnsp12 (RdRp)\x94]\x94(M)\x11M\xcc\x14e\x8c\x05nsp13\x94]\x94(M\xcd\x14M%\x17e\x8c\x05nsp14\x94]\x94(M&\x17M4\x19e\x8c\x05nsp15\x94]\x94(M5\x19M\x8e\x1ae\x8c\x05nsp16\x94]\x94(M\x8f\x1aM\xb8\x1beu\x8c\rgene_overlaps\x94}\x94(\x8c\x07exclude\x94]\x94(]\x94(\x8c\x05ORF1a\x94\x8c\x06ORF1ab\x94e]\x94(\x8c\x05ORF7a\x94\x8c\x05ORF7b\x94ee\x8c\x06retain\x94]\x94]\x94(\x8c\x01N\x94\x8c\x05ORF9b\x94eau\x8c\x13fitness_pseudocount\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\x12min_expected_count\x94K\n\x8c\x14clade_corr_min_count\x94J \xa1\x07\x00\x8c\x19aa_fitness_init_ref_clade\x94\x8c\x0319A\x94\x8c!aa_fitness_heatmap_minimal_domain\x94]\x94(J\xfa\xff\xff\xffK\x02e\x8c\x0eclade_synonyms\x94}\x94(\x8c\x0319A\x94\x8c\x01B\x94\x8c\x0320A\x94\x8c\x03B.1\x94\x8c\x0320B\x94\x8c\x05B.1.1\x94\x8c\x0320C\x94\x8c\x07B.1.367\x94\x8c\x0320E\x94\x8c\x07B.1.177\x94\x8c\x0320F\x94\x8c\x03D.2\x94\x8c\x0320G\x94\x8c\x05B.1.2\x94\x8c\x0320I\x94\x8c\x05Alpha\x94\x8c\x0320J\x94\x8c\x05Gamma\x94\x8c\x0321C\x94\x8c\x07Epsilon\x94\x8c\x0321F\x94\x8c\x04Iota\x94\x8c\x0321I\x94\x8c\x05Delta\x94\x8c\x0321J\x94\x8c\x05Delta\x94\x8c\x0321K\x94\x8c\x0cOmicron BA.1\x94\x8c\x0321L\x94\x8c\x0cOmicron BA.2\x94\x8c\x0322A\x94\x8c\x0cOmicron BA.4\x94\x8c\x0322B\x94\x8c\x0cOmicron BA.5\x94\x8c\x0322C\x94\x8c\x11Omicron BA.2.12.1\x94\x8c\x0322D\x94\x8c\x0fOmicron BA.2.75\x94\x8c\x0322E\x94\x8c\x0cOmicron BQ.1\x94u\x8c\x0cdms_datasets\x94hJ\x8c\rneher_fitness\x94\x8c\x19data/Neher_aa_fitness.csv\x94\x8c\x14clade_fixed_muts_ref\x94\x8c\x0319A\x94\x8c\x14terminal_pseudocount\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\x19terminal_min_actual_count\x94K\x05\x8c\x04dnds\x94\x8c^https://raw.githubusercontent.com/spond/SARS-CoV-2-variation/master/windowed-sites-fel-all.csv\x94\x8c\x08docs_url\x94\x8c-https://jbloomlab.github.io/SARS2-mut-fitness\x94\x8c\x15docs_plot_annotations\x94\x8c\x1fdata/docs_plot_annotations.yaml\x94u\x8c\x04rule\x94\x8c\x0canalyze_dnds\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8cP/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness');
######## snakemake preamble end #########


In [2]:
# get variables from `snakemake`
dnds_csv = snakemake.input.dnds
aa_fitness_csv = snakemake.input.aa_fitness
corr_html = snakemake.output.corr_html
min_expected_count = snakemake.params.min_expected_count

In [3]:
import altair as alt

import numpy

import pandas as pd

import scipy.stats.mstats

Get dN/dS values averaging the dN (`beta`) and dS (`alpha`) values over timeframes first, and clipping the dN/dS values at 0.05 and 20:

In [4]:
gene_map = {
    "3C": "nsp5 (Mpro)",
    "RdRp": "nsp12 (RdRp)",
    "endornase": "nsp15",
    "exonuclease": "nsp14",
    "helicase": "nsp13",
    "leader": "nsp1",
    "methyltransferase": "nsp16",
}

dnds = (
    pd.read_csv(dnds_csv)
    .query("(alpha != 0) or (beta != 0)")
    .groupby(["gene", "site"], as_index=False)
    .aggregate({"alpha": "mean", "beta": "mean"})
    .assign(
        dnds=lambda x: numpy.clip(x["beta"] / x["alpha"], a_min=0.05, a_max=20),
        log_dnds=lambda x: numpy.log(x["dnds"]),
        gene=lambda x: x["gene"].map(lambda g: gene_map[g] if g in gene_map else g),
    )
    .rename(columns={"beta": "dN", "log_dnds": "log dN/dS"})
    .drop(columns=["alpha", "dnds"])
)

dnds

,gene,site,dN,log dN/dS
0,nsp5 (Mpro),1,1.108253,1.177224
1,nsp5 (Mpro),2,1.369948,0.887329
2,nsp5 (Mpro),3,0.340120,-0.978962
3,nsp5 (Mpro),4,1.140580,-0.488341
4,nsp5 (Mpro),5,2.353438,1.495519
...,...,...,...,...
9408,nsp9,109,3.373902,2.995732
9409,nsp9,110,0.339979,-1.458756
9410,nsp9,111,0.684415,0.711573
9411,nsp9,112,0.609013,-2.394275


Read amino-acid fitnesses, and for each site:
 - the mean fitness of mutations
 - the number of effective amino-acids when amino acids are assigned a probability weight of $e^{fitness}$

In [5]:
aa_fitness = (
    pd.read_csv(aa_fitness_csv)
    .query("gene not in ['ORF1a', 'ORF1ab']")
    .query("expected_count >= @min_expected_count")
    .rename(columns={"aa_site": "site"})
    .assign(
        p_aa=lambda x: numpy.exp(x["fitness"]),
        p=lambda x: x["p_aa"] / x.groupby(["gene", "site"])["p_aa"].transform("sum"),
    )
    .groupby(["gene", "site"], as_index=False)
    .aggregate(
        mean_fitness=pd.NamedAgg("fitness", "mean"),
        fitness_entropy=pd.NamedAgg("p", lambda p: -(p * numpy.log(p)).sum()),
    )
    .assign(fitness_n_effective=lambda x: numpy.exp(x["fitness_entropy"]))
    .rename(columns={"mean_fitness": "mean fitness", "fitness_entropy": "fitness entropy"})
)

assert not (set(dnds["gene"].unique()) - set(aa_fitness["gene"].unique()))

aa_fitness

,gene,site,mean fitness,fitness entropy,fitness_n_effective
0,E,1,-4.044917,0.309668,1.362972
1,E,2,-2.786233,0.774795,2.170148
2,E,3,-1.758716,1.171519,3.226890
3,E,4,-1.636612,1.097318,2.996120
4,E,5,-1.096513,1.234616,3.437060
...,...,...,...,...,...
9736,nsp9,109,-2.159867,0.743339,2.102945
9737,nsp9,110,-2.573740,0.744499,2.105387
9738,nsp9,111,-3.732329,0.571722,1.771315
9739,nsp9,112,-2.238080,0.949155,2.583527


Merge amino-acid fitness estimates aggregated at site level with dN/dS values:

In [6]:
df = aa_fitness.merge(dnds, on=["gene", "site"], validate="one_to_one")

Get correlations for each gene, then plot:

In [7]:
dnds_metrics = ["dN", "log dN/dS"]
fitness_metrics = ["mean fitness", "fitness entropy"]

gene_corrs = (
    df
    .melt(
        id_vars=[c for c in df.columns if c not in dnds_metrics],
        value_vars=dnds_metrics,
        var_name="dnds_metric",
        value_name="dnds_value",
    )
    .groupby(["gene", "dnds_metric"])
    [["dnds_value"] + fitness_metrics]
    .corr()
    .reset_index()
    .query("level_2 == 'dnds_value'")
    .melt(
        id_vars=["gene", "dnds_metric"],
        value_vars=fitness_metrics,
        var_name="fitness metric",
        value_name="Pearson correlation",
    )
)

gene_corrs

,gene,dnds_metric,fitness metric,Pearson correlation
0,E,dN,mean fitness,0.436386
1,E,log dN/dS,mean fitness,0.227921
2,M,dN,mean fitness,0.492178
3,M,log dN/dS,mean fitness,0.318985
4,N,dN,mean fitness,0.553152
...,...,...,...,...
87,nsp7,log dN/dS,fitness entropy,0.320153
88,nsp8,dN,fitness entropy,0.117486
89,nsp8,log dN/dS,fitness entropy,0.272405
90,nsp9,dN,fitness entropy,0.165345


Now plot the gene correlations:

In [8]:
# sort by correlation
gene_order = (
    gene_corrs
    .query("dnds_metric == 'log dN/dS'")
    .groupby("gene")
    .aggregate({"Pearson correlation": "mean"})
    .sort_values("Pearson correlation", ascending=False)
    .index
    .tolist()
)

gene_corr_chart = (
    alt.Chart(gene_corrs)
    .encode(
        x=alt.X("gene", scale=alt.Scale(domain=gene_order)),
        y=alt.Y("Pearson correlation"),
        color=alt.Color("fitness metric"),
        shape=alt.Shape("fitness metric"),
        tooltip=[
            alt.Tooltip(c, format=".2f") if gene_corrs[c].dtype == float else c
            for c in gene_corrs.columns
        ],
        facet=alt.Facet(
            "dnds_metric",
            columns=1,
            sort="descending",
            title=None,
            header=alt.Header(labelFontStyle="bold", labelFontSize=13, labelPadding=2),
        ),
    )
    .mark_point(filled=True, size=50)
    .properties(width=alt.Step(25), height=175)
)

gene_corr_chart

alt.Chart(...)

In [9]:
?alt.Facet

Init signature:
alt.Facet(
    shorthand=Undefined,
    aggregate=Undefined,
    align=Undefined,
    band=Undefined,
    bin=Undefined,
    bounds=Undefined,
    center=Undefined,
    columns=Undefined,
    field=Undefined,
    header=Undefined,
    sort=Undefined,
    spacing=Undefined,
    timeUnit=Undefined,
    title=Undefined,
    type=Undefined,
    **kwds,
)
Docstring:     
Facet schema wrapper

Mapping(required=[shorthand])

Attributes
----------

shorthand : string
    shorthand for field, aggregate, and type
aggregate : :class:`Aggregate`
    Aggregation function for the field (e.g., ``"mean"``, ``"sum"``, ``"median"``,
    ``"min"``, ``"max"``, ``"count"`` ).

    **Default value:** ``undefined`` (None)

    **See also:** `aggregate <https://vega.github.io/vega-lite/docs/aggregate.html>`__
    documentation.
align : anyOf(:class:`LayoutAlign`, :class:`RowColLayoutAlign`)
    The alignment to apply to grid rows and columns. The supported string values are
    ``"all"``, ``"e